# Project 2: Housing Violations and Rodent Inspections in New York City

In New York City, complaints about poor housing conditions, rodents, and other pests are part of everyday life. Two city agencies look at different sides of this problem:

- **Housing Maintenance Code violations**, issued by the Department of Housing Preservation and Development (HPD) when apartments or buildings do not meet basic standards  
- **Rodent inspections**, carried out by the Department of Health and Mental Hygiene (DOHMH) in response to rat and pest complaints  

These two sets of records are related but not identical. A borough can have a high number of housing violations but relatively fewer rodent inspections, or the opposite. I wanted a clear, data driven view of how these patterns compare across the five boroughs.

## Main question

The main question for my project is how do the five NYC boroughs compare in **2024** in terms of the number of **housing maintenance code violations** recorded by HPD and the number of **rodent inspections** conducted by DOHMH; and what does the relationship between the two look like when they are shown together?

In other words:

> Do boroughs with more housing code violations also see more rodent inspections, and which boroughs appear to receive more rodent inspection attention relative to their volume of violations?

## Data and sources

I work with two datasets from NYC Open Data:

1. **Housing Maintenance Code Violations**  
   - Source: NYC Open Data  
   - URL: `https://data.cityofnewyork.us/resource/wvxf-dwi5.csv`  
   - Each row represents a single violation issued for a building.  
   - I use the `inspectiondate` column to select records from the **2024 calendar year** and summarize them by `boro`.

2. **Rodent Inspection**  
   - Source: NYC Open Data  
   - URL: `https://data.cityofnewyork.us/resource/p937-wjvj.csv`  
   - Each row represents a single rodent inspection at a specific location.  
   - I use the `inspection_date` column to keep only inspections from **2024** and summarize them by `borough`.

Both datasets are loaded directly from their CSV API endpoints with `pandas.read_csv`, and all cleaning, filtering, and aggregation is done in Python.

## Brief overview of what I do in this notebook

1. Load each dataset from its NYC Open Data.  
2. Convert the inspection date columns to datetime and filter to **2024**.  
3. Aggregate each dataset to get **one row per borough** with:
   - total housing violations in 2024  
   - total rodent inspections in 2024  
4. Merge the two borough level summaries into a single combined table.  
5. Create two visualizations:
   - a **scatter plot** showing the relationship between violations and inspections by borough  
   - a **bar chart** of rodent inspections **per 10,000 housing violations** by borough  
6. Conclude with an interpretation section that discusses the main patterns and some limitations of this simple analysis.


In [133]:
# ensure the visualizations render properly across VSCode, Jupyter Book, etc.
# https://plotly.com/python/renderers/

import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

## Setup

In this section I import the Python libraries I use throughout the project and set up my notebook.

- `pandas` for loading, cleaning, and aggregating the data.  
- `plotly.express` for interactive charts.


In [134]:
import pandas as pd
import plotly.express as px


Here I load the **Housing Maintenance Code Violations** dataset from NYC Open Data
into a DataFrame called `hpd_raw`. I also check the shape and column names to make
sure the data loaded correctly.

In [135]:
# HPD Housing Maintenance Code Violations CSV API URL
hpd_url = "https://data.cityofnewyork.us/resource/wvxf-dwi5.csv"

hpd_url


'https://data.cityofnewyork.us/resource/wvxf-dwi5.csv'

In [136]:
hpd_raw = pd.read_csv(hpd_url)

hpd_raw.head()

,violationid,buildingid,registrationid,boroid,boro,housenumber,lowhousenumber,highhousenumber,streetname,streetcode,...,violationstatus,rentimpairing,latitude,longitude,communityboard,councildistrict,censustract,bin,bbl,nta
0,10081311,375411,306067,3,BROOKLYN,22 FRONT,22 FRON,22 FRONT,STAGG STREET,80930,...,Close,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10299683,375411,306067,3,BROOKLYN,22 FRONT,22 FRON,22 FRONT,STAGG STREET,80930,...,Close,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10299685,375411,306067,3,BROOKLYN,22 FRONT,22 FRON,22 FRONT,STAGG STREET,80930,...,Close,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10299686,375411,306067,3,BROOKLYN,22 FRONT,22 FRON,22 FRONT,STAGG STREET,80930,...,Open,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10299690,375411,306067,3,BROOKLYN,22 FRONT,22 FRON,22 FRONT,STAGG STREET,80930,...,Open,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here I list all the columns in `hpd_raw` to see which fields are available and to identify the specific columns I want to use. The sample confirms that `inspectiondate` and `boro` look usable for filtering and grouping.


In [137]:
hpd_raw.columns.to_list()

['violationid',
 'buildingid',
 'registrationid',
 'boroid',
 'boro',
 'housenumber',
 'lowhousenumber',
 'highhousenumber',
 'streetname',
 'streetcode',
 'zip',
 'apartment',
 'story',
 'block',
 'lot',
 'class',
 'inspectiondate',
 'approveddate',
 'originalcertifybydate',
 'originalcorrectbydate',
 'newcertifybydate',
 'newcorrectbydate',
 'certifieddate',
 'ordernumber',
 'novid',
 'novdescription',
 'novissueddate',
 'currentstatusid',
 'currentstatus',
 'currentstatusdate',
 'novtype',
 'violationstatus',
 'rentimpairing',
 'latitude',
 'longitude',
 'communityboard',
 'councildistrict',
 'censustract',
 'bin',
 'bbl',
 'nta']

The original dataset is very large so I add a `$limit=500000` parameter to ensure I get a large but manageable subset of the data and then check the overall shape of the table.

In [138]:
hpd_url = "https://data.cityofnewyork.us/resource/wvxf-dwi5.csv?$limit=500000"

hpd_raw = pd.read_csv(hpd_url)
hpd_raw.shape

/var/folders/l_/cmp00n796rv699g42dndf6w80000gn/T/ipykernel_83949/436861827.py:3: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



(500000, 41)

I define convenient variables for the HPD columns I plan to use:

- `borough_col` for the borough column (`"boro"`)  
- `date_col` for the inspection date column (`"inspectiondate"`)

In [139]:
borough_col = "boro"            # borough column
date_col = "inspectiondate"     # date column 

borough_col, date_col

('boro', 'inspectiondate')

Here, I make a working copy of the HPD table (`hpd`); convert the inspection date column to a proper datetime (`pd.to_datetime`); and filter to keep only inspections that took place in the year **2024**.

The result is a filtered DataFrame `hpd_2024` that I use for the rest of the analysis.

In [140]:
# make a working copy of the big table
hpd = hpd_raw.copy()

# convert inspectiondate to real datetime
hpd[date_col] = pd.to_datetime(hpd[date_col], errors="coerce")

# filter to 2024 only
hpd_2024 = hpd[hpd[date_col].dt.year == 2024].copy()

# see how big it is
hpd_2024.shape

(185142, 41)

To avoid re-downloading and re-filtering every time I run the notebook, I save the 2024 HPD violations data to `data/hpd_violations_2024.csv`.

In [141]:
hpd_2024.to_csv("data/hpd_violations_2024.csv", index=False)

I reloaded the saved 2024 HPD dataset from disk and quickly checked the first few rows to confirm that the saved file looks correct.

In [142]:
hpd_2024 = pd.read_csv("data/hpd_violations_2024.csv")

hpd_2024.head()

,violationid,buildingid,registrationid,boroid,boro,housenumber,lowhousenumber,highhousenumber,streetname,streetcode,...,violationstatus,rentimpairing,latitude,longitude,communityboard,councildistrict,censustract,bin,bbl,nta
0,17441822,395065,303005,3,BROOKLYN,2840,2840,2840,WEST 36 STREET,92080,...,Close,N,40.576042,-74.001991,13.0,47.0,328.0,3188693.0,3.070040e+09,Coney Island-Sea Gate
1,17441823,395065,303005,3,BROOKLYN,2840,2840,2840,WEST 36 STREET,92080,...,Close,N,40.576042,-74.001991,13.0,47.0,328.0,3188693.0,3.070040e+09,Coney Island-Sea Gate
2,17257767,876647,224764,2,BRONX,31,31,43,WEST TREMONT AVENUE,76850,...,Close,Y,40.851777,-73.911335,5.0,14.0,243.0,2114452.0,2.028690e+09,University Heights (South)-Morris Heights
3,17257917,324060,300267,3,BROOKLYN,116,114,128,LENOX ROAD,55330,...,Close,N,40.653818,-73.955329,17.0,40.0,822.0,3116644.0,3.050840e+09,East Flatbush-Erasmus
4,17257918,324060,300267,3,BROOKLYN,116,114,128,LENOX ROAD,55330,...,Close,N,40.653818,-73.955329,17.0,40.0,822.0,3116644.0,3.050840e+09,East Flatbush-Erasmus


Next I work with the **Rodent Inspection** dataset from the Department of Health and Mental Hygiene (DOHMH).  
This dataset contains one row per inspection and includes the inspection date and borough.

In [143]:
# Rodent Inspection dataset
rats_url = "https://data.cityofnewyork.us/resource/p937-wjvj.csv?$limit=200000"

rats_raw = pd.read_csv(rats_url)

rats_raw.shape

(200000, 25)

I load the rodent inspections dataset into a DataFrame called `rats_raw` and use `.head()` to preview the first few rows and verify that the load was successful.

In [144]:
rats_raw.head()
rats_raw.columns.tolist()

['inspection_type',
 'job_ticket_or_work_order_id',
 'job_id',
 'job_progress',
 'bbl',
 'boro_code',
 'block',
 'lot',
 'house_number',
 'street_name',
 'zip_code',
 'x_coord',
 'y_coord',
 'latitude',
 'longitude',
 'borough',
 'inspection_date',
 'result',
 'approved_date',
 'location',
 'community_board',
 'council_district',
 'census_tract',
 'bin',
 'nta']


Here, I define variables for the rodent inspection dataset:

- `rats_borough_col` for the borough column (`"borough"`)  
- `rats_date_col` for the inspection date column (`"inspection_date"`)

These were helpful for me in cleaning and summarizing the data.

In [145]:
rats_borough_col = "borough"
rats_date_col    = "inspection_date"

rats_borough_col, rats_date_col

('borough', 'inspection_date')

I make a working copy of the rodent inspection data (`rats_2024` starts from a cleaned copy); convert the rodent `inspection_date` column to datetime; and filter to keep only inspections from the year **2024**.

This gives me a `rats_2024` DataFrame that matches the same year as the HPD data.

In [146]:
# make a working copy of the raw rats data
rats = rats_raw.copy()

# convert the inspection_date column to real datetime
rats[rats_date_col] = pd.to_datetime(rats[rats_date_col], errors="coerce")

# keep only 2024 rows
rats_2024 = rats[rats[rats_date_col].dt.year == 2024].copy()

rats_2024.shape

(3280, 25)

This step is to confirm that `inspection_date` and `borough` columns from `rats_2024` were parsed correctly and that the borough values look reasonable.

In [147]:
rats_2024[[rats_date_col, rats_borough_col]].head()

,inspection_date,borough
10,2024-07-26 10:25:45,Bronx
110,2024-04-24 13:20:00,Brooklyn
156,2024-05-06 12:09:53,Manhattan
181,2024-08-16 11:14:51,Manhattan
194,2024-01-29 14:33:24,Manhattan


Similar to HPD, I save the filtered 2024 rodent inspections to `data/rats_2024.csv` so I do not need to repeat the filtering every time I run the notebook.

In [148]:
rats_2024.to_csv("data/rats_2024.csv", index=False)

Now, I reload the saved 2024 HPD violations (`hpd_2024`) and 2024 rodent inspections (`rats_2024`) from their CSV files, and briefly inspect them to confirm they look correct before summarizing.

In [149]:
hpd_2024  = pd.read_csv("data/hpd_violations_2024.csv")
rats_2024 = pd.read_csv("data/rats_2024.csv")

hpd_2024.head()
rats_2024.head()


,inspection_type,job_ticket_or_work_order_id,job_id,job_progress,bbl,boro_code,block,lot,house_number,street_name,...,borough,inspection_date,result,approved_date,location,community_board,council_district,census_tract,bin,nta
0,Initial,13910623,PC8356981,1,2.031480e+09,2,3148,25,2489,TIEBOUT AVENUE,...,Bronx,2024-07-26 10:25:45,Rat Activity,2024-07-29T09:36:17.000,"\n, \n(40.861687018452, -73.894991108705)",5.0,15.0,39902.0,2013480.0,Fordham Heights
1,Initial,13847424,PC8313842,1,3.034430e+09,3,3443,54,323,MOFFAT STREET,...,Brooklyn,2024-04-24 13:20:00,Passed,2024-04-25T14:57:00.000,"\n, \n(40.691148604503, -73.902887376384)",4.0,37.0,409.0,3080063.0,Bushwick (East)
2,Compliance,13855593,PC8309159,2,1.019620e+09,1,1962,38,1201,AMSTERDAM AVENUE,...,Manhattan,2024-05-06 12:09:53,Passed,2024-05-08T12:44:15.000,"\n, \n(40.808831287107, -73.95957386795)",9.0,7.0,20701.0,1059510.0,Morningside Heights
3,Initial,13924645,PC8391531,1,1.020630e+09,1,2063,37,404,WEST 149 STREET,...,Manhattan,2024-08-16 11:14:51,Passed,2024-08-19T15:22:57.000,"\n, \n(40.826758592034, -73.943364257844)",9.0,9.0,231.0,1061518.0,Hamilton Heights-Sugar Hill
4,Compliance,13791611,PC8223539,2,1.016890e+09,1,1689,27,2287,1 AVENUE,...,Manhattan,2024-01-29 14:33:24,Failed for Other R,2024-01-30T09:03:01.000,"\n, \n(40.797047586417, -73.93465283911)",11.0,8.0,188.0,1079359.0,East Harlem (North)


To compare boroughs, I collapse the HPD 2024 data to **one row per borough**.  
I group by `boro` and count the number of violations, creating the summary table `hpd_by_boro` with a column `hpd_violations_2024`.

In [150]:
hpd_by_boro = (
    hpd_2024
    .groupby("boro", as_index=False)["violationid"]
    .count()
    .rename(columns={"violationid": "hpd_violations_2024"})
)

hpd_by_boro

,boro,hpd_violations_2024
0,BRONX,56893
1,BROOKLYN,67306
2,MANHATTAN,24018
3,QUEENS,32638
4,STATEN ISLAND,4287


I do the same type of aggregation for the rodent data:  
group the 2024 rodent inspections by `borough` and count the number of inspections, creating the summary table `rats_by_boro` with a column `rat_inspections_2024`.

In [151]:
rats_by_boro = (
    rats_2024
    .groupby("borough", as_index=False)["job_id"]
    .count()
    .rename(columns={"job_id": "rat_inspections_2024"})
)

rats_by_boro

,borough,rat_inspections_2024
0,Bronx,621
1,Brooklyn,1367
2,Manhattan,1046
3,Queens,217
4,Staten Island,29


HPD and DOHMH store borough names in slightly different formats (`boro` vs `borough`, and capitalization differs).  
Here I create a shared `Borough` column in both summary tables using a consistent title case format (e.g., `"Bronx"`, `"Brooklyn"`) so I can merge them reliably.

In [152]:
# Making a formatted borough name in BOTH tables
hpd_by_boro["Borough"]  = hpd_by_boro["boro"].str.title()
rats_by_boro["Borough"] = rats_by_boro["borough"].str.title()

I merge `hpd_by_boro` and `rats_by_boro` on the common `Borough` column. This produces a single `combined` table with **one row per borough** containing:

- total HPD housing violations in 2024 (`hpd_violations_2024`)  
- total rodent inspections in 2024 (`rat_inspections_2024`)

In [153]:
combined = hpd_by_boro.merge(
    rats_by_boro,
    on="Borough",       
    how="inner"
)
combined

,boro,hpd_violations_2024,Borough,borough,rat_inspections_2024
0,BRONX,56893,Bronx,Bronx,621
1,BROOKLYN,67306,Brooklyn,Brooklyn,1367
2,MANHATTAN,24018,Manhattan,Manhattan,1046
3,QUEENS,32638,Queens,Queens,217
4,STATEN ISLAND,4287,Staten Island,Staten Island,29


Now I select only the columns I need for visualization:

- `Borough`  
- `hpd_violations_2024`  
- `rat_inspections_2024`

and store them in a cleaner DataFrame called `combined_clean`.

In [154]:
combined_clean = combined[["Borough", "hpd_violations_2024", "rat_inspections_2024"]].copy()

combined_clean


,Borough,hpd_violations_2024,rat_inspections_2024
0,Bronx,56893,621
1,Brooklyn,67306,1367
2,Manhattan,24018,1046
3,Queens,32638,217
4,Staten Island,4287,29


## Visual 1 – Scatter plot: Violations vs Rodent Inspections

This scatter plot compares the **raw counts** for each borough:

- x axis: total HPD housing violations in 2024  
- y axis: total DOHMH rodent inspections in 2024  
- Each point represents one borough and is labeled with the borough name

This lets me see whether boroughs with more housing violations also tend to have more rodent inspections.

In [155]:
fig = px.scatter(
    combined_clean,
    x="hpd_violations_2024",
    y="rat_inspections_2024",
    text="Borough",
    hover_name="Borough",
    labels={
        "hpd_violations_2024": "Housing Code Violations (HPD, 2024)",
        "rat_inspections_2024": "Rodent Inspections (DOHMH, 2024)",
    },
    title="Housing Violations vs. Rodent Inspections by Borough, NYC 2024"
)

fig.update_traces(textposition="top center")

fig.show()

### Interpreting the scatter plot

- Boroughs with more housing violations generally also have more rodent inspections.  
- Brooklyn and the Bronx sit high on both axes, indicating a heavy burden of both violations and inspections.  
- Manhattan has fewer violations than Brooklyn and the Bronx but still a relatively high number of inspections.  
- Queens and Staten Island have lower counts of both violations and inspections.

This view shows how the two datasets move together, but it does not show inspection *intensity* relative to violations. For that, I compute a normalized rate.


## Visual 2 – Rat inspections per 10,000 housing violations

To compare inspection intensity across boroughs more fairly, I calculate a simple rate:

> rodent inspections per 10,000 HPD violations

for each borough. This shows how many rodent inspections DOHMH conducts relative to the number of housing violations, not just the raw counts.


In [156]:
combined_ratio = combined_clean.copy()

combined_ratio["rats_per_10k_violations"] = (
    combined_ratio["rat_inspections_2024"]
    / combined_ratio["hpd_violations_2024"]
    * 10000
)

combined_ratio

,Borough,hpd_violations_2024,rat_inspections_2024,rats_per_10k_violations
0,Bronx,56893,621,109.152268
1,Brooklyn,67306,1367,203.102249
2,Manhattan,24018,1046,435.506703
3,Queens,32638,217,66.486917
4,Staten Island,4287,29,67.646373


I sort the boroughs by this rate and keep a tidy version of the table with:

- `Borough`  
- `hpd_violations_2024`  
- `rat_inspections_2024`  
- `rats_per_10k_violations`

This makes it easier to see which boroughs receive more rodent inspections per violation.

In [157]:
combined_ratio_sorted = combined_ratio.sort_values(
    "rats_per_10k_violations", ascending=False
)

combined_ratio_sorted[["Borough", "hpd_violations_2024",
                       "rat_inspections_2024", "rats_per_10k_violations"]]

,Borough,hpd_violations_2024,rat_inspections_2024,rats_per_10k_violations
2,Manhattan,24018,1046,435.506703
1,Brooklyn,67306,1367,203.102249
0,Bronx,56893,621,109.152268
4,Staten Island,4287,29,67.646373
3,Queens,32638,217,66.486917


Finally, I plot the **rat inspections per 10,000 housing violations** as a bar chart by borough.  
This highlights which boroughs have higher or lower inspection intensity relative to their housing violation counts.

In [158]:
import plotly.express as px

fig2 = px.bar(
    combined_ratio_sorted,
    x="Borough",
    y="rats_per_10k_violations",
    labels={
        "Borough": "Borough",
        "rats_per_10k_violations": "Rodent inspections per 10,000 HPD violations (2024)",
    },
    title="Rat Inspections Relative to Housing Violations, NYC 2024",
)

fig2.update_layout(
    yaxis_tickformat=".1f",   # e.g., 15.2 instead of long decimals
)

fig2.show()

## Overall takeaways and limitations

**Takeaways**

1. Housing violations and rodent inspections are positively related: boroughs with more violations generally also have more rodent inspections.  
2. Brooklyn and the Bronx carry a heavy burden in absolute numbers, with many violations and many inspections.  
3. When I normalize by violations, Manhattan appears to have a higher rate of rodent inspections per 10,000 violations than the outer boroughs.  
4. A simple borough level merge of HPD and DOHMH data already reveals differences in how housing conditions and pest control are addressed across the city.

**Limitations**

This analysis has several limitations. First, it only uses one year of data (2024), so the patterns I see might look different if I examined multiple years or longer term trends. Second, I work with counts of records, not counts of unique buildings or households, which means some addresses may appear many times while others do not appear at all. I also do not adjust for population size, the amount of housing stock, or neighborhood level characteristics, so comparisons across boroughs are not normalized for how many people or units are actually at risk. 
Finally, both reporting behavior and enforcement practices likely differ by borough, which can influence the number of violations recorded and the number of inspections conducted, independent of the true underlying conditions.

## Note on the use of AI tools

I used an ChatGPT to support parts of this project. At the beginning, I used it to help define and narrow the scope of the project and to sanity check whether my research question about housing violations and rodent inspections made sense for the assignment requirements. 
I also asked for help with some coding details, such as how to filter by year using the inspection date columns, how to group and aggregate by borough, and how to structure the Plotly code for the scatter plot and bar chart. Finally, I also took assistance from AI to doublecheck on my use of graphs and if they best show the relationship between the two datasets and to improve the clarity of my markdown explanations.

